In [1]:
import numpy as np
import pandas as pd

In [2]:
transactions_df = pd.read_excel('data/raw/SIBS_Analytics_Freguesias_detalhado_20220818.xlsx')

In [3]:
transactions_df.head()

,Ano/Mês,Tipo de dia,País do cartão,Setor,Freguesia,Nº de operações,Valor das operações
0,Mai 22,Dia da Semana,Andorra,Restaurantes,ARU/ORU do Centro Histórico,15,356.05
1,Mai 22,Dia da Semana,Andorra,Restaurantes,"União das freguesias de Cedofeita, Santo Ildef...",29,780.76
2,Mai 22,Dia da Semana,Andorra,"Bares, Cafés e Similares","União das freguesias de Cedofeita, Santo Ildef...",20,467.45
3,Mai 22,Dia da Semana,Angola,"Tecnologia, Electrónica e Electrodomésticos","União das freguesias de Cedofeita, Santo Ildef...",17,2572.55
4,Mai 22,Dia da Semana,Angola,Vestuário,"União das freguesias de Cedofeita, Santo Ildef...",63,8865.15


In [4]:
def replace_origin_local(row):
    if row['País do cartão'] == 'Portugal':
        return 'local'
    else:
        return 'non-local'

In [5]:
transactions_df['origin'] = transactions_df.apply(lambda row: replace_origin_local(row), axis=1)

In [53]:
transactions_monthly_df = transactions_df.groupby(['origin', 'Setor', 'Freguesia',
                                                   'Ano/Mês'], as_index=False).sum()

In [123]:
def calculate_ratio(row, transactions_df):
    """
    Calculates probability of operation being done by a local
    or non-local in a given sector, freguesia, and date
    """
    setor = row['Setor']
    freguesia = row['Freguesia']
    date = row['Ano/Mês']

    if row['origin'] == 'local':
        reverse_origin = 'non-local'
    else:
        reverse_origin = 'local'

    condition = (transactions_df['origin'] == reverse_origin) & (transactions_df['Setor'] == setor) & \
            (transactions_df['Freguesia'] == freguesia) & (transactions_df['Ano/Mês'] == date)

    try:
        reverse_origin_transactions = transactions_monthly_df[condition]['Nº de operações'].values[0]
    except IndexError:
        reverse_origin_transactions = 0

    ratio = row['Nº de operações']/(row['Nº de operações'] + reverse_origin_transactions)

    # print(ratio)

    return ratio

In [124]:
transactions_monthly_df['ratio'] = transactions_monthly_df.apply(lambda row: calculate_ratio(row, transactions_monthly_df), axis=1)

In [129]:
transactions_monthly_df[(transactions_monthly_df['origin'] == 'non-local') & \
                        (transactions_monthly_df['Setor'] == 'Restaurantes')]

,origin,Setor,Freguesia,Ano/Mês,Nº de operações,Valor das operações,ratio
1124,non-local,Restaurantes,ARU/ORU da Corujeira,Jul 22,1608,67578.02,0.132063
1125,non-local,Restaurantes,ARU/ORU da Corujeira,Jun 22,1187,46975.58,0.107295
1126,non-local,Restaurantes,ARU/ORU da Corujeira,Mai 22,1085,41679.47,0.103471
1127,non-local,Restaurantes,ARU/ORU de Campanhã - Estação,Jul 22,5816,91336.93,0.272502
1128,non-local,Restaurantes,ARU/ORU de Campanhã - Estação,Jun 22,4555,69118.96,0.243245
1129,non-local,Restaurantes,ARU/ORU de Campanhã - Estação,Mai 22,3700,56628.19,0.209181
1130,non-local,Restaurantes,ARU/ORU do Centro Histórico,Jul 22,154286,4698585.74,0.582754
1131,non-local,Restaurantes,ARU/ORU do Centro Histórico,Jun 22,133183,4021480.08,0.565070
1132,non-local,Restaurantes,ARU/ORU do Centro Histórico,Mai 22,127921,4050854.31,0.543988
1133,non-local,Restaurantes,Bonfim,Jul 22,7985,222069.94,0.112948
